# 作業 : (Kaggle)鐵達尼生存預測

In [1]:
# 載入套件與資料
import pandas as pd
import numpy as np

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [2]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [5]:
print (df.dtypes)
print ('')
print (df.columns)

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')


In [6]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

In [7]:
# FOR INT_FEATURES
mean_int = pd.DataFrame( df[int_features].mean())
max_int = pd.DataFrame( df[int_features].max() )
nunique_int = pd.DataFrame( df[int_features].nunique() )
int_result = pd.concat([mean_int,max_int,nunique_int],axis=1)
int_result.columns = ['mean', 'max','nunique']
int_result

,mean,max,nunique
Pclass,2.294882,3,3
SibSp,0.498854,8,7
Parch,0.385027,9,8


In [8]:
# FOR FLOAT_FEATURES
mean_float = pd.DataFrame( df[float_features].mean())
max_float = pd.DataFrame( df[float_features].max() )
nunique_float = pd.DataFrame( df[float_features].nunique() )
float_result = pd.concat([mean_float,max_float,nunique_float],axis=1)
float_result.columns = ['mean', 'max','nunique']
float_result

,mean,max,nunique
Age,29.881138,80.0000,98
Fare,33.295479,512.3292,281


In [9]:
# FOR OBJECT_FEATURES
mean_object = pd.DataFrame( df[object_features].mean())
max_object = pd.DataFrame( df[object_features].max() )
nunique_object = pd.DataFrame( df[object_features].nunique() )
object_result = pd.concat([mean_object,max_object,nunique_object],axis=1)
object_result.columns = ['mean', 'max','nunique']
object_result

C:\Users\kevin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,mean,max,nunique
Cabin,NaN,NaN,186
Embarked,NaN,NaN,3
Name,NaN,"van Melkebeke, Mr. Philemon",1307
Sex,NaN,male,2
Ticket,NaN,WE/P 5735,929


Object Features的三種操作將會出現問題。由於其為類別資料，Mean將不會出現數值 ； 某些類別的Nunique沒有意義 ；Max出現謬誤